In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import random
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-10-29 11:34:16.567078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

In [14]:

display(pd.DataFrame({
    "A": train_a.describe()["pv_measurement"], 
    "B": train_b.describe()["pv_measurement"], 
    "C": train_c.describe()["pv_measurement"]
    }))

,A,B,C
count,34085.000000,32844.000000,26095.000000
mean,630.594707,96.824778,77.631060
min,0.000000,-0.000000,-0.000000
25%,0.000000,-0.000000,0.000000
50%,5.940000,0.000000,0.000000
75%,678.040000,96.600000,58.800000
max,5733.420000,1152.300000,999.600000
std,1165.902422,193.946487,165.816877


pv measurement for location A is very different from B and C

In [15]:
display(pd.DataFrame({
    "A": X_train_observed_a.mean(), 
    "B": X_train_observed_b.mean(), 
    "C": X_train_observed_c.mean()
    }))

display(pd.DataFrame({
    "A": X_train_observed_a.std(), 
    "B": X_train_observed_b.std(), 
    "C": X_train_observed_c.std()
    }))


,A,B,C
date_forecast,2021-02-09 23:30:00.000000512,2020-09-01 00:00:00.000000256,2020-08-31 11:00:00.000000512
absolute_humidity_2m:gm3,6.652576,6.125604,6.013866
air_density_2m:kgm3,1.249064,1.254262,1.253214
ceiling_height_agl:m,2907.949219,2651.450439,2605.747559
clear_sky_energy_1h:J,599401.75,542151.4375,542358.3125
clear_sky_rad:W,166.500381,150.59758,150.655075
cloud_base_agl:m,1737.142212,1731.706665,1710.178467
dew_or_rime:idx,0.022398,0.008484,-0.013867
dew_point_2m:K,276.823395,275.524536,275.1474
diffuse_rad:W,44.918282,40.894447,40.920612


,A,B,C
date_forecast,356 days 20:14:00.496045568,351 days 14:39:35.235266020,351 days 07:09:15.242667696
absolute_humidity_2m:gm3,2.796349,2.727944,2.78984
air_density_2m:kgm3,0.036319,0.035289,0.038392
ceiling_height_agl:m,2570.413086,2404.853516,2415.11792
clear_sky_energy_1h:J,879542.375,840052.3125,840484.3125
clear_sky_rad:W,244.945358,233.948273,234.069214
cloud_base_agl:m,1815.584106,1726.011108,1751.569336
dew_or_rime:idx,0.190367,0.117327,0.369077
dew_point_2m:K,6.701763,6.773766,7.103381
diffuse_rad:W,64.786697,61.530506,61.696342


Perhaps training models for each location is beneficial?